In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
from collections import Counter
import matplotlib.pyplot as plt
import powerlaw,math
import os
import scipy.stats as stats
import random
from numpy import polyfit, poly1d

In [2]:
pa_year_neuro = pd.read_csv('PaperId_Year_chemistry.csv')
# citing_cited = pd.read_csv('citing_cited_chemistry.csv')
# neuro_er_mag = pd.read_csv('chemistry_er_mag.csv')
pa_au = pd.read_csv('paper_author_chemistry.csv')  #mag中原始的 paper author 对应
pa_au_matched = pd.read_csv('PaperId_PID_chemistry.csv')
neuro = pd.read_csv('chemistry_1206.csv')
neuro_mentee = neuro.MenteeID[neuro.StopYear<2000]
neuro_mentor = neuro.MentorID[neuro.StopYear<2000]
neuro_er = list(pd.concat([neuro_mentee,neuro_mentor]).drop_duplicates())

In [3]:
with open('Citations.pickle','rb')as f:
    Citations = pickle.load(f)

In [4]:
pub_Info = {}
count = 0
for k in neuro_er[0:10]:
    print(count)
    pas_k = pa_au_matched.PaperId[pa_au_matched.PID==k]   # all papers of author k in MAG  1
    year_pas_k = np.array(pa_year_neuro.Year[pa_year_neuro.PaperId.isin(pas_k)]) # year of papers abovel
    if len(year_pas_k)>0:
        the_first_year = min(year_pas_k) # 2
        the_last_year = max(year_pas_k)
        career_len = the_last_year - the_first_year # 3
        yearly_productivity = len(pas_k)/career_len # 4
        # 5
        tp = year_pas_k - the_first_year
        py_pas_k = np.array(pa_year_neuro.PaperId[pa_year_neuro.PaperId.isin(pas_k)]) # papers having Year
        theFirst5Year_pubs = py_pas_k[tp<=5]  # 5 

        # citation
    #     tc = citing_cited.cited[citing_cited.cited.isin(pas_k)]
    #     total_c = len(tc)                # 6 
        total_c = 0
        for pk in pas_k:
            try:
                tpc = Citations[pk]
            except:
                tpc = 0
            total_c+=tpc

        yearly_c = total_c/career_len    # 7
        # 8 
    #     tc5 = citing_cited.cited[citing_cited.cited.isin(theFirst5Year_pubs)]
        tc5 = 0
        for pk in theFirst5Year_pubs:
            try:
                tpc = Citations[pk]
            except:
                tpc = 0
            tc5+=tpc

        theFirst5Year_c = tc5      # 8
    
        #coauthor
        coau = pa_au.AuthorID[pa_au.PaperId.isin(pas_k)].drop_duplicates() #不重复的合作者  #9
        yearly_coau = len(coau)/career_len  # 10
        theFirst5Year_coau = pa_au.AuthorID[pa_au.PaperId.isin(theFirst5Year_pubs)].drop_duplicates() # 11

        #受训期间的导师发表情况
        traintion = {} #循环的是每个学生和其受训时间   # 11
        year_train = np.array(neuro[['MenteeID','StopYear']][neuro.MentorID==k].drop_duplicates())
        if len(year_train)>0:
            for ii in range(len(year_train)):
                year_pas_k1 = pd.DataFrame(year_pas_k)
                py_pas_k1 = pd.DataFrame(py_pas_k)

                tx = year_train[ii]
                yy = tx[1]  #上面学生受训的时间
                yt = py_pas_k1[0][(year_pas_k1[0]>=yy-3)&(year_pas_k1[0]<yy+3)]  #受训时期前后总共6年内的文章 11

                #找在期间学生与导师的合作情况
                ym = tx[0]  #具体受训的学生
                coauthered = np.array(pa_au_matched.PID[pa_au_matched.PaperId.isin(yt)])
                cn = len(coauthered[coauthered==ym])   # 12 受训期间该学生与导师的合作文章数量

                traintion[ym] = (len(yt),cn)           #key是受训的每个学生     
        else:
            traintion = {}
    
        pub_Info[k] = [len(pas_k),the_first_year,career_len,yearly_productivity,len(theFirst5Year_pubs),
                       total_c,yearly_c,theFirst5Year_c,len(coau),yearly_coau,len(theFirst5Year_coau),
                      traintion]
    else:
        pub_Info[k] = []
    count+=1
# 0.总发文量
# 1.出道年
# 2.生涯长度
# 3.年均文章数
# 4.前5年的发文量
# 5.总引用
# 6.年均引用
# 7.前五年内的总引用
# 8.总合作者数量
# 9.年均合作者数量
# 10.前5年的合作者数量
# 11.训练期间的文章发表与合作情况 

0
1
2
3
4
5
6
7
8
9


In [5]:
with open('pub_Info0_10000.pickle','wb') as f:
    pickle.dump(pub_Info,f)

{18761: [276,
  1986.0,
  35.0,
  7.885714285714286,
  19,
  15879,
  453.6857142857143,
  745,
  37,
  1.0571428571428572,
  5,
  {18760: (68, 0),
   20259: (0, 0),
   37982: (0, 0),
   10786: (0, 0),
   54383: (41, 8),
   49078: (0, 0),
   4166: (19, 4),
   119255: (50, 0),
   663417: (0, 0),
   679452: (0, 0),
   780205: (39, 0)}],
 53604: [91,
  1995.0,
  26.0,
  3.5,
  3,
  5466,
  210.23076923076923,
  165,
  5,
  0.19230769230769232,
  1,
  {}],
 20: [60,
  1985.0,
  36.0,
  1.6666666666666667,
  6,
  5263,
  146.19444444444446,
  327,
  14,
  0.3888888888888889,
  2,
  {390: (0, 0),
   566: (0, 0),
   1260: (11, 0),
   4558: (11, 0),
   8871: (0, 0),
   8872: (16, 0),
   2397: (0, 0),
   15295: (12, 0),
   18688: (17, 0),
   55454: (11, 2),
   56612: (0, 0),
   102419: (6, 0),
   102420: (19, 0),
   688612: (0, 0),
   740458: (8, 0)}],
 12561: [134,
  1972.0,
  38.0,
  3.526315789473684,
  11,
  3837,
  100.97368421052632,
  305,
  25,
  0.6578947368421053,
  4,
  {60556: (0, 0